In [1]:
# To start this notebook with more than one thread, change the extension setting in vs, search for thread,
# and change the Max Threads as well as settings.json.
# This will speed up the computations for running multiple chains. 

# Ensure that Julia was launched with an appropriate number of threads
println(Threads.nthreads())
nr_chains = 4 # number of chains to sample, corresponding to the threads

4


4

In [2]:
# using Pkg
# Pkg.add("Turing")
using Plots # to plot things
using Random, Distributions # generate random noise
using LinearAlgebra # identity matrix
using Turing # sampling package
using StatsPlots
using StatsBase # for defining customized distributions 

using QuadGK
using BasicBSpline
using LaTeXStrings

using Optim # for ML and MAP estimation 

using SparseArrays

include("../src/Mesh_fun.jl")
include("../src/FE_fun.jl")
include("../src/SIACMatrix.jl")


PostProcess (generic function with 1 method)

In [3]:
# define test function
# testFun(x)=@. 2*x*sin(pi*x)
testFun(x)=@. sin(2*pi*x)+0.5*cos(4pi*x)^2 + 0.5

# define parameters
# N = 100 # number of points
# Define the list of N values
N_values = [100, 200, 400, 800] # Note: due to the choice of SIAC parameters, we need at least 52 points.

# variance of the iid Gaussian noise added to the measurements
σ² = 0.05 
# Piecewise Polynomial Setup for SIAC Filter
p = 3

# Dictionary to store results for each N
results = Dict()

for N in N_values
    println("Generating data for N = $N")

    # generate true physical data on the uniform grid
    xx = collect(LinRange(0,1,N))
    fx = testFun(xx) 

    # forward operator, noise, and noisy data
    # forward operator -- use the identity matrix for now
    F = Matrix(1.0I, N, N)  # Identity matrix of Float64 type
    Random.seed!(123) # Setting the seed
    noise = rand( Normal(0,sqrt(σ²)), N )
    fx_noisy = F * fx + noise

    # SIAC matrix
    # Create SIAC Matrix, with periodic boundary treatment
    Ffil = global_SIAC_Mat(p, xx, boundaryTreatment="periodic")
    # filtered noisy data
    fx_noisy_filter = Ffil*fx_noisy

    # Store results
    results[N] = (xx, fx, F, fx_noisy, Ffil, fx_noisy_filter)
end

println("All data has been generated!")

Generating data for N = 100
Generating data for N = 200
Generating data for N = 400
Generating data for N = 800
All data has been generated!


# Set up the statistical model 

We consider the following model: 
\begin{align*}
    \mathbf{fx\_noisy} & \sim \mathcal{N}(F\mathbf{u},\sigma^2*I) = \mathcal{N}(F\mathbf{u},\alpha^{-1}*I), \\
    R \mathbf{u} & \sim \mathcal{N}(\mathbf{0},\beta^{-1}*I), \\ 
\end{align*}
where $R$ is the regularization operator. 
The associated posterior log PDF is 
$$            
                -\frac{\alpha}{2}  \| F\mathbf{u} - \mathbf{fx\_noisy} \|_2^2 
                -\frac{\beta}{2}  \| R\mathbf{u} \|_2^2 
                - d \alpha - d\beta + (\frac{N}{2}+c-1)\log \alpha + (\frac{N}{2}+c-1)\log \beta.
$$

In [4]:
# Set up the statistical model

for N in N_values
    println("Setting up statistical model for N = $N")

    # Retrieve stored data
    xx, fx, F, fx_noisy, Ffil, fx_noisy_filter = results[N]

    # regularization operator
    R = Matrix(1.0I, N, N) - Ffil

    c = 1
    d = 1.0e-3

    # Store results
    results[N] = (xx, fx, F, fx_noisy, Ffil, fx_noisy_filter, R, c, d)
end

# Define the posterior log-PDF for both unknown signal and hyperparameters  
function logpdf_posterior(z, F, fx_noisy, R, c, d, N) 
    u, α, β = z[1:end-2], z[end-1], z[end]

    if α<=0 || β<=0
        logpdf = -Inf
    else
        logpdf = -(.5α)*norm(F*u-fx_noisy)^2 -(.5β)*norm(R*u)^2 - d*α - d*β + (N/2+c-1)*log(α) + (N/2+c-1)*log(β)
    end
    
    return logpdf
end

println("All statistical models are set up!")

Setting up statistical model for N = 100
Setting up statistical model for N = 200
Setting up statistical model for N = 400
Setting up statistical model for N = 800
All statistical models are set up!


# MAP estimate

We will compare the MAP estimates obtained from the built-in optimizer (limited-memory Broyden–Fletcher–Goldfarb–Shanno L-BFGS algorithm) and the Block Coordinate Descent (BCD) algorithm.

## L-BFGS

In [5]:
# Dictionary to store MAP estimates
map_results_LBFGS = Dict()

for N in N_values
    println("Computing MAP estimate using LBFGS for N = $N")
    
    # Retrieve stored values
    xx, fx, F, fx_noisy, Ffil, fx_noisy_filter, R, c, d = results[N]

    # Initial guess for the optimization (u, α and β)
    initial_guess = vcat(zeros(N), 1.0, 1.0)

    # Define a function handle for Optim.jl
    neg_logpdf(z) = -logpdf_posterior(z, F, fx_noisy, R, d, c, N)

    # Obtain MAP estimate using built-in optimizer
    # Measure execution time
    tLBFGS = @elapsed begin # execution time
        # MAP estimation
        map_estimate = optimize(neg_logpdf, initial_guess, LBFGS())
    end

    # Extract the MAP estimate of u
    fx_LBFGS_tmp = Optim.minimizer(map_estimate)
    fx_LBFGS = fx_LBFGS_tmp[1:end-2]

    # Store results
    map_results_LBFGS[N] = (fx_LBFGS, tLBFGS)
end

println("All MAP estimations are completed!")

Computing MAP estimate using LBFGS for N = 100
Computing MAP estimate using LBFGS for N = 200
Computing MAP estimate using LBFGS for N = 400
Computing MAP estimate using LBFGS for N = 800
All MAP estimations are completed!


## BCD

In [6]:
# set up BCD function
using SparseArrays

function BCD_1d_scalar(F, y, R, c, d, QUIET)

    t_start = time()  # Measure time

    # Global constants and defaults
    MIN_ITER = 0
    MAX_ITER = 1000
    ABSTOL = 1e-8
    RELTOL = 1e-4

    # Data preprocessing
    m, n = size(F)  # number of (indirect) measurements and pixels
    k = size(R, 1)  # number of outputs of the regularization operator
    FtF = sparse(F' * F)  # product corresponding to the forward operator
    Fty = F' * y  # forward operator applied to the indirect data

    # Initial values for the inverse variances and the mean and others
    aalpha = 0.1
    bbeta = 1
    mu = zeros(n)
    mu_OLD = zeros(n)  # mean

    C_inv = Matrix(1.0I, n, n)

    abs_error = 0
    rel_error = 0

    if !QUIET
        println("iter\t abs error\t abs tol\t rel error\t rel tol")
    end

    # Iterate between the update steps until convergence or max number of iterations
    for counter in 1:MAX_ITER

        # 1) Fix aalpha, bbeta, and update x
        C_inv = sparse(aalpha * FtF + bbeta * R' * R)  # update covariance matrix
        mu = C_inv \ (aalpha * Fty)  # update the mean

        # 2) Fix x, B and update aalpha
        aalpha = (m + 2 * c) / (norm(F * mu - y)^2 + 2 * d)

        # 3) Fix x, aalpha and update B
        bbeta = (n + 2 * c) / (norm(R * mu)^2 + 2 * d)

        # Store certain values in history structure
        abs_error = norm(mu - mu_OLD)^2  # absolute error
        rel_error = (norm(mu - mu_OLD) / norm(mu_OLD))^2  # relative error

        mu_OLD = mu  # store value of mu

        if !QUIET
            println("$(counter)\t $(abs_error)\t $(ABSTOL)\t $(rel_error)\t $(RELTOL)")
        end

        # Check for convergence
        if abs_error < ABSTOL && rel_error < RELTOL && counter > MIN_ITER
            break
        end

    end

    # Output the time it took to perform all operations
    if !QUIET
        println("Elapsed time: ", time() - t_start, " seconds")
    end

    return mu, C_inv, aalpha, bbeta, Dict("abs_error" => abs_error, "rel_error" => rel_error)
end

BCD_1d_scalar (generic function with 1 method)

In [7]:
# Dictionary to store MAP estimates using BCD
map_results_BCD = Dict()

for N in N_values
    println("Computing MAP estimate using BCD for N = $N")

    # Retrieve stored values
    xx, fx, F, fx_noisy, Ffil, fx_noisy_filter, R, c, d = results[N]

    # Obtain MAP estimate using BCD algorithm
    tBCD = @elapsed begin # execution time
        # Perform MAP estimation using BCD algorithm
        fx_BCD, C_inv, aalpha, bbeta, history = BCD_1d_scalar(F, fx_noisy, R, c, d, false ); 
    end

    # Store results
    map_results_BCD[N] = (fx_BCD, C_inv, aalpha, bbeta, history, tBCD)
end

println("All MAP estimations (BCD) are completed!")

Computing MAP estimate using BCD for N = 100
iter	 abs error	 abs tol	 rel error	 rel tol
1	 108.01713463921439	 1.0e-8	 Inf	 0.0001
2	 0.1008690037317874	 1.0e-8	 0.0009338241017843853	 0.0001
3	 0.06835619187844917	 1.0e-8	 0.0006342583531601523	 0.0001
4	 0.0034393450745363874	 1.0e-8	 3.195162123908988e-5	 0.0001
5	 4.167835195122321e-6	 1.0e-8	 3.872637420001169e-8	 0.0001
6	 2.8338742803050538e-9	 1.0e-8	 2.6331746901783367e-11	 0.0001
Elapsed time: 0.050492048263549805 seconds
Computing MAP estimate using BCD for N = 200
iter	 abs error	 abs tol	 rel error	 rel tol
1	 212.56719834860053	 1.0e-8	 Inf	 0.0001
2	 0.19204608196529052	 1.0e-8	 0.0009034605689742577	 0.0001
3	 0.14050942152622448	 1.0e-8	 0.0006624095445607871	 0.0001
4	 0.013119063817657288	 1.0e-8	 6.192454425669237e-5	 0.0001
5	 1.2604163289943729e-5	 1.0e-8	 5.9511687269737245e-8	 0.0001
6	 4.842696817394352e-9	 1.0e-8	 2.2865421513420173e-11	 0.0001
Elapsed time: 0.028203964233398438 seconds
Computing MAP estimat

In [8]:
using Printf

# Print table header
println("\nExecution Time & Relative L2 Error Summary")
println("===========================================================================================")
println(@sprintf("%-10s %-15s %-15s %-15s %-15s %-15s", "N", "Time (LBFGS)", "Time (BCD)", "Rel l2 Err (LBFGS)", "Rel l2 Err (BCD)", "Rel l2 Err (SIAC)"))
println("-------------------------------------------------------------------------------------------")

# Loop over N values to compute errors and print results
for N in N_values
    # Retrieve execution times
    time_LBFGS = map_results_LBFGS[N][2]  # Time for LBFGS method
    time_BCD = map_results_BCD[N][6]      # Time for BCD method

    # Retrieve true and estimated solutions
    fx_true = results[N][2]  # True solution fx
    fx_LBFGS = map_results_LBFGS[N][1]  # Optim estimate
    fx_BCD = map_results_BCD[N][1]    # BCD estimate

    # Compute relative L2 errors
    rel_l2_err_LBFGS = norm(fx_LBFGS - fx_true) / norm(fx_true)
    rel_l2_err_BCD = norm(fx_BCD - fx_true) / norm(fx_true)

    xx, fx, F, fx_noisy, Ffil, fx_noisy_filter = results[N]
    rel_l2_err_SIAC = norm(fx_noisy_filter - fx_true) / norm(fx_true)

    # Print table row
    println(@sprintf("%-10d %-15.1e %-15.1e %-15.1e %-15.1e %-15.1e", N, time_LBFGS, time_BCD, rel_l2_err_LBFGS, rel_l2_err_BCD, rel_l2_err_SIAC))
end

println("===========================================================================================")


Execution Time & Relative L2 Error Summary
N          Time (LBFGS)    Time (BCD)      Rel l2 Err (LBFGS) Rel l2 Err (BCD) Rel l2 Err (SIAC)
-------------------------------------------------------------------------------------------
100        7.1e-01         2.3e+00         1.3e-01         4.8e-02         7.2e-02        
200        5.2e-01         2.8e-02         1.4e-01         6.4e-02         8.7e-02        
400        1.0e+01         1.2e-02         9.5e-02         5.5e-02         9.3e-02        
800        1.4e+02         6.2e-02         8.6e-02         5.6e-02         9.0e-02        
